# Goal
1. Test the streamflow classes that I created and make sure they are working.
2. Create a clean demo notebook to commit.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("/home/nick/C2S-Python-API/")
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shapely
import rasterio
import dataretrieval.nwis as nwis # If missing install with !pip install dataretrieval
import time

In [ ]:
from c2s.hydrology.stream_gauges import StreamGaugeRasterizer

# Setup

In [ ]:
# test geometry to collect stream gauges for
bbox = (-96.29352194405351, 34.209756945451005, -95.065592721268, 35.15411842539479)

# find the stream gauges for this bbox
daily_or_subdaily_values = "dv" 
discharge_or_height = "00060" # "00060": discharge, "00065": height

df_nwis, meta_nwis = nwis.what_sites(
    bBox=np.round(bbox, 6).tolist(), # truncate to avoid 400 error!
    outputDataTypeCd=daily_or_subdaily_values,
    parameterCd=discharge_or_height # discharge parameter
    )
df_nwis["years_of_history"] = df_nwis[['begin_date', 'end_date']].apply(lambda x: (pd.to_datetime(x['end_date']) - pd.to_datetime(x['begin_date'])).days / 365, axis=1)

In [ ]:
# get the DV values for a particular day
site_ids = df_nwis["site_no"].tolist()
dv_gdf, _ = nwis.get_dv(sites=site_ids, parameterCd="00060", start="2023-05-01", end="2023-05-01")
dv_gdf

In [ ]:
# need to add the coordinates for these stations
dv_gdf = dv_gdf.reset_index().merge(df_nwis[["site_no", "dec_lat_va", "dec_long_va"]], on="site_no", how="left")
dv_gdf

In [ ]:
# rasterization properties; in practice, these may be derived from an actual raster
# here, we are just coding them for convenience
width = 350  # measured in pixels
height = 350
transform = rasterio.transform.from_bounds(*bbox, width, height)

# Rasterization examples

In [ ]:
# initialize rasterizer
rasterizer = StreamGaugeRasterizer(
    dv_gdf,
    "00060_Mean",
    transform,
    350,
    350,
    lat_col="dec_lat_va",
    lon_col="dec_long_va",
)

## No interpolation

In [ ]:
# create a raster with no interpolation
start = time.time()
no_interp = rasterizer.points_to_raster_no_interpolation()
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(no_interp)
plt.title("Stream gauges, no interpolation")
plt.show()

In [ ]:
# doesn't look like much, but we can show that it adds up to the same value as the original table
print(np.sum(dv_gdf.drop_duplicates()["00060_Mean"]))
print(np.sum(no_interp))

## Rasterize points with a buffer
This makes much better physical sense if the data is in a projected CRS so that the buffer distance is in meters instead of degrees, but it works either way

In [ ]:
# create a raster with no interpolation
start = time.time()
no_interp = rasterizer.buffered_points_to_raster(buffer_dist=0.01)
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(no_interp)
plt.title("Stream gauges, buffered")
plt.show()

## Simple interpolation
Just interpolate between all of the stream gauge values. Supported methods are `nearest`, `linear`, and `cubic`.

In [ ]:
start = time.time()
nearest_interp = rasterizer.naive_interpolation(interp_method="nearest")
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(nearest_interp)
plt.title("Stream gauges, nearest interpolation")
plt.show()

In [ ]:
# this method cannot interpolate outside of the minimum envelope of the gauges; see wall-to-wall method below
start = time.time()
linear_interp = rasterizer.naive_interpolation(interp_method="linear")
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(linear_interp)
plt.title("Stream gauges, linear interpolation")
plt.show()

In [ ]:
# this method cannot interpolate outside of the minimum envelope of the gauges; see wall-to-wall method below
start = time.time()
cubic_interp = rasterizer.naive_interpolation(interp_method="cubic")
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(cubic_interp)
plt.title("Stream gauges, cubic interpolation")
plt.show()

## Wall-to-wall interpolation
This is a simple approach to creating a wall-to-wall interpolation. First, the nearest interpolation is performed. The corner values are extracted from this interpolation. These corners are then included along with the original stream gauge values when doing a linear or cubic interpolation.

In [ ]:
start = time.time()
full_interp = rasterizer.naive_interpolation_wall_to_wall(interp_method="cubic")
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(full_interp)
plt.title("Stream gauges, wall-to-wall interpolation")
plt.show()

Soon, we will probably want to create a more logical/sophisticated version of this which uses stream gauges from outside the AOI in order to create a smoother raster without edge artifacts.

## Inverse distanced weighted interpolation
This method builds off the wall-to-wall interpolation. After creating that interpolation, a second array is create internally which is simply the number of pixels from any location to the nearest gauge. The inverse of those distances are used to weight the interpolated array.

In [ ]:
start = time.time()
idw_interp = rasterizer.inverse_distance_weighted_interpolation(interp_method="cubic", distance_exp=1)
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(idw_interp)
plt.title("Stream gauges, wall-to-wall interpolation")
plt.show()

The inverse distance array can cause the values to drop off too quickly; raising this array to an exponent on (0, 1) will make it drop off more slowly as the distance from gauges increases. This drop-off is applied by default (exponent of 0.20).

In [ ]:
start = time.time()
idw_interp = rasterizer.inverse_distance_weighted_interpolation(interp_method="cubic")
print(f"Elapsed time: {time.time() - start:0.4f} sec")

plt.imshow(idw_interp)
plt.title("Stream gauges, wall-to-wall interpolation")
plt.show()

## HAND weighted

In [ ]:
# initialize rasterizer
rasterizer = StreamGaugeRasterizer(
    dv_gdf,
    "00060_Mean",
    transform,
    350,
    350,
    lat_col="dec_lat_va",
    lon_col="dec_long_va",
)

In [ ]:
start = time.time()
hand_weighted = rasterizer.hand_weighted_interpolation(
    bbox, 
    "/data/static/hand/mosaic_250m_avg_EPSG4269.tif",
    wbt_path="/home/nick"
)
print(f"Elapsed time: {time.time() - start:0.4f} sec")

In [ ]:
plt.imshow(np.ma.masked_less(hand_weighted, 0))
plt.title("HAND-weighted interpolation")
plt.show()